# Transfer Learning

#### Models used for high-level feature extraction

 **Model**         | **Size (MB)** | **Top-1 Accuracy** | **Top-5 Accuracy** | **Parameters** | **Depth** | **Time (ms) per inference step (CPU)** | **Time (ms) per inference step (GPU)** 
------------------:|--------------:|-------------------:|-------------------:|---------------:|----------:|---------------------------------------:|---------------------------------------:
 InceptionV3       | 92            | 0.779              | 0.937              | 23,851,784     | 159       | 42.25                                  | 6.86                                   
 *EfficientNetB0*    | 29            | -                  | -                  | 5,330,571      | -         | 46                                     | 4.91                                   
 ResNet50          | 98            | 0.749              | 0.921              | 25,636,712     | -         | 58.2                                   | 4.55                                   
 VGG16             | 528           | 0.713              | 0.901              | 138,357,544    | 23        | 69.5                                   | 4.16                                   
 DenseNet121       | 33            | 0.75               | 0.923              | 8,062,504      | 121       | 77.14                                  | 5.38                                   
 Xception          | 88            | 0.79               | 0.945              | 22,910,480     | 126       | 109.42                                 | 8.06                                   
 InceptionResNetV2 | 215           | 0.803              | 0.953              | 55,873,736     | 572       | 130.19                                 | 10.02                                  


> Data source: https://keras.io/api/applications/#available-models  
> Table converter: https://tableconvert.com/excel-to-markdown

## Preparations

#### Import packages

In [80]:
#import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import matplotlib.pyplot as plt
#%matplotlib inline

#### Structure of `data/split` directory

```
data/split
└── 40X
    ├── test
    │   ├── adenosis
    │   ├── ductal_carcinoma
    │   ├── fibroadenoma
    │   ├── lobular_carcinoma
    │   ├── mucinous_carcinoma
    │   ├── papillary_carcinoma
    │   ├── phyllodes_tumor
    │   └── tubular_adenoma
    ├── train
    │   ├── adenosis
    │   ├── ductal_carcinoma
    │   ├── fibroadenoma
    │   ├── lobular_carcinoma
    │   ├── mucinous_carcinoma
    │   ├── papillary_carcinoma
    │   ├── phyllodes_tumor
    │   └── tubular_adenoma
    └── val
        ├── adenosis
        ├── ductal_carcinoma
        ├── fibroadenoma
        ├── lobular_carcinoma
        ├── mucinous_carcinoma
        ├── papillary_carcinoma
        ├── phyllodes_tumor
        └── tubular_adenoma
```

#### Define image generator

In [81]:
image_generator_train = ImageDataGenerator(
    #rescale=1/127.5,
    #rescale=127.5,
    #width_shift_range= 10.0,
    #height_shift_range= 10.0,
    #rotation_range = 20,
    #horizontal_flip = True,
    #vertical_flip = False,
    #zoom_range = 0.1,
    #channel_shift_range = 0.2,
    #brightness_range = (0,1),
    #shear_range = 0.2
)

In [82]:
image_generator_valtest = ImageDataGenerator(
    #rescale=1/127.5
)

In [83]:
image40Xtrain = image_generator_train.flow_from_directory(
    os.path.join('data','split','40X','train'),
    batch_size=16, 
    target_size=(460, 700),
    class_mode = 'sparse',
    shuffle=True
)

Found 1594 images belonging to 8 classes.


In [84]:
image40Xval = image_generator_valtest.flow_from_directory(
    os.path.join('data','split','40X','val'),
    batch_size=16, 
    target_size=(460, 700),
    class_mode = 'sparse',
    shuffle=True
)

Found 195 images belonging to 8 classes.


In [85]:
image40Xtest = image_generator_valtest.flow_from_directory(
    os.path.join('data','split','40X','test'),
    batch_size=16, 
    target_size=(460, 700),
    class_mode = 'sparse',
    shuffle=True
)

Found 206 images belonging to 8 classes.


#### Print shape of images and labels

In [86]:
imgs, labels = image40Xtrain.next()
print('Images:', imgs.shape)
print('Labels:', labels.shape)

Images: (16, 460, 700, 3)
Labels: (16,)


#### Print range of pixel values

In [87]:
print('lowest pixel value:',np.min(imgs), '\nhighest pixel value:', np.max(imgs))

lowest pixel value: 23.0 
highest pixel value: 255.0


#### Number of images per class for magnitude 40x

In [88]:
for imgset, imgset_title in zip([image40Xtrain, image40Xval, image40Xtest], ['train','val','test']):
    print('\n', imgset_title)
    for i in range(8):
        lb = list(imgset.class_indices)[i]
        sumclass = sum(imgset.labels==i)
        print('n:', sumclass, ', ratio:', round(sumclass/imgset.n*100), ': 40x',lb)
    print(imgset.n,': Total images with magnitude 40x',)


 train
n: 91 , ratio: 6 : 40x adenosis
n: 691 , ratio: 43 : 40x ductal_carcinoma
n: 202 , ratio: 13 : 40x fibroadenoma
n: 124 , ratio: 8 : 40x lobular_carcinoma
n: 164 , ratio: 10 : 40x mucinous_carcinoma
n: 116 , ratio: 7 : 40x papillary_carcinoma
n: 87 , ratio: 5 : 40x phyllodes_tumor
n: 119 , ratio: 7 : 40x tubular_adenoma
1594 : Total images with magnitude 40x

 val
n: 11 , ratio: 6 : 40x adenosis
n: 86 , ratio: 44 : 40x ductal_carcinoma
n: 25 , ratio: 13 : 40x fibroadenoma
n: 15 , ratio: 8 : 40x lobular_carcinoma
n: 20 , ratio: 10 : 40x mucinous_carcinoma
n: 14 , ratio: 7 : 40x papillary_carcinoma
n: 10 , ratio: 5 : 40x phyllodes_tumor
n: 14 , ratio: 7 : 40x tubular_adenoma
195 : Total images with magnitude 40x

 test
n: 12 , ratio: 6 : 40x adenosis
n: 87 , ratio: 42 : 40x ductal_carcinoma
n: 26 , ratio: 13 : 40x fibroadenoma
n: 17 , ratio: 8 : 40x lobular_carcinoma
n: 21 , ratio: 10 : 40x mucinous_carcinoma
n: 15 , ratio: 7 : 40x papillary_carcinoma
n: 12 , ratio: 6 : 40x phyllo

> **The typical transfer-learning workflow**

> 1. Instantiate a base model and load pre-trained weights into it.
> 1. Freeze all layers in the base model by setting trainable = False.
> 1. Create a new model on top of the output of one (or several) layers from the base model.
> 1. Train your new model on your new dataset.

> see [The typical transferlearning workflow](https://keras.io/guides/transfer_learning/#the-typical-transferlearning-workflow)

In [89]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

https://keras.io/guides/transfer_learning/#an-endtoend-example-finetuning-an-image-classification-model-on-a-cats-vs-dogs-dataset

#### Using random data augmentation

In [90]:
data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

In [91]:
base_model = keras.applications.EfficientNetB0(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(460, 700, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

In [92]:
# Freeze the base_model
base_model.trainable = False

In [93]:
# Create new model on top
inputs = keras.Input(shape=(460, 700, 3))
x = data_augmentation(inputs)  # Apply random data augmentation
#x = inputs

In [94]:
# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
#scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
#x = scale_layer(x)

In [95]:
# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(8)(x)
model = keras.Model(inputs, outputs)

model.summary()


Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 460, 700, 3)]     0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 460, 700, 3)       0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 15, 22, 1280)      4049571   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 10248     
Total params: 4,059,819
Trainable params: 10,248
Non-trainable params: 4,049,571
____________________________________________

#### Train the top layer

In [96]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    #loss=keras.losses.BinaryCrossentropy(from_logits=True),
    #metrics=[keras.metrics.BinaryAccuracy()],
    #optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['acc']
)

In [97]:
# End training when accuracy stops improving (optional)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

In [98]:
# Train model with subsample
image40Xtrain.reset()
train_imgs, train_lbs = image40Xtrain.next()
val_imgs, val_lbs = image40Xval.next()
epochs = 2 #20
model.fit(
    x=train_imgs, 
    y=train_lbs, 
    epochs=epochs, 
    validation_data=(val_imgs, val_lbs),
    callbacks=[early_stopping]
)

Epoch 1/2
1/1 [==============================] - 17s 17s/step - loss: 11.8864 - acc: 0.1250 - val_loss: 12.3509 - val_acc: 0.0625
Epoch 2/2
1/1 [==============================] - 9s 9s/step - loss: 10.8090 - acc: 0.1250 - val_loss: 12.2616 - val_acc: 0.0000e+00


In [99]:
model.get_layer('top_conv')

ValueError: No such layer: top_conv.

In [20]:
# Train model with whole sample
# todo: preprocess_input ?
epochs = 2 #20
history = model.fit_generator(
    generator=image40Xtrain, 
    validation_data=image40Xval, 
    epochs=epochs, 
    callbacks=[early_stopping]
)

Epoch 1/2


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/envs/exts-ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-4d2cbefef5cf>", line 7, in <module>
    callbacks=[early_stopping]
  File "/usr/local/Caskroom/miniconda/base/envs/exts-ml/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 2177, in fit_generator
    initial_epoch=initial_epoch)
  File "/usr/local/Caskroom/miniconda/base/envs/exts-ml/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_generator.py", line 176, in fit_generator
    x, y, sample_weight=sample_weight, class_weight=class_weight)
  File "/usr/local/Caskroom/miniconda/base/envs/exts-ml/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1940, in train_on_batch
    outputs = self.train_function(ins)
  File "/usr/local/Caskroom/miniconda/base/envs/exts-ml/l

TypeError: object of type 'NoneType' has no len()

#### Do a round of fine-tuning of the entire model

In [39]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 460, 700, 3)]     0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 460, 700, 3)       0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 15, 22, 1280)      4049571   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 10248     
Total params: 4,059,819
Trainable params: 4,017,796
Non-trainable params: 42,023
____________________________________________

In [40]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    #loss=keras.losses.BinaryCrossentropy(from_logits=True),
    #metrics=[keras.metrics.BinaryAccuracy()],
    loss='sparse_categorical_crossentropy',
    metrics=['acc']
)

In [ ]:
#train_imgs, train_labels = image40Xtrain.next()
#val_imgs, val_labels = image40Xval.next()
#X_tr = preprocess_input(train_imgs)
#X_val = preprocess_input(val_imgs)

epochs = 2 #10
model.fit(
    x=train_imgs, 
    y=train_lbs, 
    epochs=epochs, 
    validation_data=(val_imgs, val_lbs),
    callbacks=[early_stopping]
)

Epoch 1/2


In [ ]:
# Train model with whole sample
# todo: preprocess_input ?
epochs = 2 #20
history = model.fit_generator(
    generator=image40Xtrain, 
    validation_data=image40Xval, 
    epochs=epochs, 
    callbacks=[early_stopping]
)

#### First, instantiate a base model with pre-trained weights.

In [9]:
# define model
# I use average pooling here, may change during process
base_model = Xception(
    weights='imagenet', 
    include_top=False, 
    input_shape=(460,700,3), 
    #pooling='avg' #'max'
)

#### Then, freeze the base model.

In [10]:
base_model.trainable = False

#### Create a new model on top.

In [11]:
inputs = keras.Input(shape=(460, 700, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)


#### Train the model on new data.

In [12]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
model.fit(new_dataset, epochs=20, callbacks=..., validation_data=...)


AttributeError: module 'tensorflow._api.v1.keras.losses' has no attribute 'BinaryCrossentropy'

#### This is how to implement fine-tuning of the whole base model: